<a href="https://colab.research.google.com/github/leiderTic/Proyecto-Almacen/blob/main/Arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

class datasetIngestadoArima():
  def __init__(self, dataframe):
        self.df = dataframe
        self.df['Date'] = pd.to_datetime(self.df['Date'], dayfirst=True)
        self.df.set_index('Date', inplace=True)
        self.df.sort_index(inplace=True, ascending=True)
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        self.model = None
        self.modelo_entrenado = None

  def test_stationarity(self, n_diff):
    """
    Método para probar la estacionariedad de una serie temporal.
    Parámetros:
    - n_diff: int, número de diferenciaciones a realizar.
    Imprime el resultado del test de Dickey-Fuller y muestra un gráfico de la serie temporal con la media móvil y desviación estándar móvil.
    """
    # Realizar la diferenciación si n_diff es distinto de 0
    if n_diff != 0:
        # Almacenar la serie diferenciada en una variable auxiliar
        df_diff = self.df.diff(n_diff)
    else:
        # Si n_diff es 0, no realizar ninguna diferenciación
        df_diff = self.df
    # Calcular la media móvil y la desviación estándar móvil de la serie diferenciada
    MA = df_diff['Total'].rolling(window=12).mean()
    MSTD = df_diff['Total'].rolling(window=12).std()
    # Graficar la serie temporal, la media móvil y la desviación estándar móvil
    plt.figure(figsize=(15,5))
    orig = plt.plot(df_diff['Total'], color='blue',label='Original')
    mean = plt.plot(MA, color='red', label='Media móvil')
    std = plt.plot(MSTD, color='black', label='Std rodante')
    plt.legend(loc='best')
    plt.title('Media móvil y Desviación estándar')
    plt.show(block=False)
    # Realizar el test de Dickey-Fuller y mostrar los resultados
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(df_diff['Total'].dropna(), autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

  def tsplot(self, n_diff, lags=None, figsize=(12, 7), style='bmh'):
    """
    Método para mostrar gráficos de análisis de series temporales.
    Parámetros:
    - n_diff: int, número de diferenciaciones a realizar.
    - lags: int, cantidad de lags a incluir en los gráficos ACF y PACF.
    - figsize: tuple, tamaño de la figura.
    - style: str, estilo de la figura.
    Muestra gráficos de la serie temporal, ACF y PACF.
    """
    # Realizar la diferenciación si n_diff es distinto de 0
    if n_diff != 0:
        # Almacenar la serie diferenciada en una variable auxiliar
        df_diff = self.df.diff(n_diff)
        df_diff = df_diff.dropna()
    else:
        # Si n_diff es 0, no realizar ninguna diferenciación
        df_diff = self.df
    # Configurar el estilo de la figura
    with plt.style.context(style):
        # Crear la figura y las subgráficas
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        # Graficar la serie temporal
        df_diff['Total'].plot(ax=ts_ax)
        # Realizar el test de Dickey-Fuller y mostrar el resultado en el título del gráfico
        p_value = sm.tsa.stattools.adfuller(df_diff['Total'].dropna())[1]
        ts_ax.set_title('Gráficos de análisis de series temporales\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        # Graficar el ACF y PACF de la serie diferenciada
        smt.graphics.plot_acf(df_diff['Total'], lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(df_diff['Total'], lags=lags, ax=pacf_ax)
        plt.tight_layout()

  def fit_arima(self, order):
        """
        Método para ajustar un modelo ARIMA a los datos.
        Parámetros:
        - order: tuple, especificación del orden del modelo ARIMA (p, d, q).
        Retorna:
        - model_fit: objeto ARIMAResultsWrapper, modelo ARIMA ajustado a los datos.
        """
        model = ARIMA(self.df['Total'], order=order)
        model_fit = model.fit()
        self.modelo_entrenado = model_fit
        print(model_fit.summary())

  def predict_arima(self, order, test_size):
        """
        Método para realizar predicciones con un modelo ARIMA ajustado.
        Parámetros:
        - model_fit: objeto ARIMAResultsWrapper, modelo ARIMA ajustado a los datos.
        - test_size: int, tamaño del conjunto de prueba para realizar predicciones.

        Retorna:
        - predictions_series: pandas Series, serie temporal con las predicciones realizadas.
        """
        size = len(self.df) - test_size
        self.train, self.test = self.df['Total'][:size], self.df['Total'][size:]
        history = [x for x in self.train]
        predictions = []
        for t in range(len(self.test)):
            model = ARIMA(history, order=order)
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(float(yhat))
            obs = self.test[t]
            history.append(obs)
            print('predicted = %f, expected = %f' % (yhat, obs))
        predictions_series = pd.Series(predictions, index=self.test.index)
        self.plot_predictions(predictions_series)
        self.plot_predictionsAux(predictions_series)
         # Crear DataFrame con las predicciones y nombrar las columnas
        predictions_df = pd.DataFrame(predictions_series, columns=['ARIMA'])
        return predictions_df

# Método para descomponer estacionalmente y visualizar los componentes
  def seasonal_decompose_plot(self, period, model='multiplicative'):
        """
        Método para realizar descomposición estacional y visualizar los componentes.
        Parámetros:
        - period: int, período de la descomposición estacional.
        - model: str, modelo de descomposición ('additive' o 'multiplicative').
        Muestra gráficos de los componentes observados, de tendencia, estacionales y residuales.
        """
        # Extraer la columna total
        total_column = self.df['Total']
        # Ajustar el tamaño de la figura y crear subgráficas
        fig, axes = plt.subplots(4, 1, figsize=(12, 10))
        # Llamar a seasonal_decompose y plotear cada componente por separado
        result = sm.tsa.seasonal_decompose(total_column, period=period, model=model)
        result.observed.plot(ax=axes[0], legend=False)
        axes[0].set_ylabel('Observado')
        result.trend.plot(ax=axes[1], legend=False)
        axes[1].set_ylabel('Tendencia')
        result.seasonal.plot(ax=axes[2], legend=False)
        axes[2].set_ylabel('Estacional')
        # Calcular y visualizar residuos manualmente
        residuals = total_column - result.trend - result.seasonal
        residuals.plot(ax=axes[3], legend=False)
        axes[3].set_ylabel('Residuales ')
        # Ajustar el espacio entre subgráficas
        plt.tight_layout()
        plt.show()

  def plot_predictions(self,predictions_series):
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
        plt.subplot(1, 1, 1)
        # Ajustar el rango de x_range para que tenga la misma longitud que self.train
        x_range = range(len(self.train))
        y_range = range(len(self.train), len(self.train) + len(predictions_series))
        ax.tick_params(axis='x', labelsize=20)
        ax.tick_params(axis='y', labelsize=20)
        # Graficar el historial con el rango modificado en el eje x
        plt.plot(x_range, self.train, label='Historial')
        # Graficar las predicciones con el rango modificado en el eje x
        plt.plot(y_range, predictions_series, label='Predicción')
        plt.legend(fontsize=20)
        plt.legend

  def plot_predictionsAux(self,predictions_series):
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
        plt.subplot(1, 1, 1)
        # Ajustar el rango de x_range para que tenga la misma longitud que self.train
        x_range = range(len(self.df['Total']))
        y_range = range(len(self.train), len(self.train) + len(predictions_series))
        ax.tick_params(axis='x', labelsize=20)
        ax.tick_params(axis='y', labelsize=20)
        # Graficar el historial con el rango modificado en el eje x
        plt.plot(x_range, self.df['Total'], label='Historial')
        # Graficar las predicciones con el rango modificado en el eje x
        plt.plot(y_range, predictions_series, label='Predicción')
        plt.legend(fontsize=20)
        plt.legend